**Работу выполнили:**

**Ванюшков Максим 381908-1**

**Иванов Аркадий  381908-1**

Ссылка на таблицу: https://docs.google.com/spreadsheets/d/1biJm2dIHUHlJPWWEdj6sjOTv4TlAMtmS/edit?usp=sharing&ouid=100458953741102245072&rtpof=true&sd=true

Написать программу, рассчитывающую затраты на онлайн университет и на оффлайн университет.

На сайте ННГУ выложены ваши учебные планы - бакалавриата и магистратуры. За основу можно взять их, выкинув всякие бесполезные по вашему мнению предметы.

И конечно можно варьировать количество часов лекций и практик.

Количество студентов в группе - параметр, равный 12 или 22.

Один преподаватель ведет n пар в неделю, n = 5 или 10 ( ставка и полставки). Один преподаватель не может вести больше 3 х разных предметов.

Начисления на оплату труда 30.1 % (платит работодатель ) до вычитания подоходного налога.

Плюс оплата интернета.
Обязательно содержание двух бухгалтеров, расходы на 1С, системного администратора.

Плюс пару методистов, которые будут отбиваться от министерства образования во время проверок и аккредитации.

К оффлайн образованию прибавляем стоимость содержания здания (или аренда помещений), коммуналка, охрана, уборщица, гардеробщица, стулья столы, раз в пять лет обновление компьютеров, обновление проекторов,, закупка канцелярии, бумаги, антисептика.

Зарплаты берите средние с hh.ru

Учитывайте инфляцию, можно взять официальную, хотя реальная обычно больше.

Результатом должны стать цифры сколько каждый студент должен заплатить если учится онлайн и оффлайн. И сколько лет должен учиться. В бакалавриате и в магистратуре.

Не забывайте, что не смотря на то, что учитесь вы не весь год подряд, зарплату всем работникам нужно платить круглый год.

Результаты расчетов - код и краткий отчет выкладывайте на гитхаб или гугл док. И открываете мне доступ.

На гитхабе я - Belokamenskaya
Почта - getmanskaya.alexandra@gmail.com

In [1]:
#подключение библиотеки для считывания данных с таблицы и математической библиотеки
import xlrd
import math

In [2]:
#подключение Google Drive
from google.colab import drive
drive.mount('/content/drive', force_remount = True)

Mounted at /content/drive


В функции *openexcel* мы открываем таблицу на Google Drive по адресу *url*, выбираем лист *(sheet)*, считываем таблицу в аргумент *vals* и возвращаем данные таблицы.

In [3]:
def openexcel(url, page):
  rb = xlrd.open_workbook(url)
  sheet = rb.sheet_by_index(page)
  vals = [sheet.row_values(rownum) for rownum in range(sheet.nrows)]
  return vals

Классы *Teacher* и *Subject* использовались для удобства обращения к данным.

In [4]:
class Teacher():
  def __init__(self, subjectName, salary):
    self.subjectName = subjectName
    self.salary = int(salary)
  
  def __repr__(self):
    return self.subjectName + ' - ' + str(self.salary) + ' руб.\n'

In [5]:
class Subject():
  def __init__(self, name, lecture, practice, laboratory, semester):
    self.name = name
    self.lecture = int(lecture)
    self.practice = int(practice)
    self.laboratory = int(laboratory)
    self.semester = int(semester)

  def __repr__(self):
    text = self.name + '\n  '
    if self.lecture > 0:
      text+= 'lectures: ' + str(self.lecture) + '\n  '
    if self.practice > 0:
      text+= 'practices: ' + str(self.practice) + '\n  '
    if self.laboratory > 0:
      text+= 'laboratories: ' + str(self.laboratory) + '\n  '
    text+= 'semesters: ' + str(self.semester) + '\n'
    return text

Функция *countTeachers* подсчитывает необходимое количество преподавателей на полную ставку и на полставки для каждого предмета.

In [6]:
def countTeachers(subjects, countGroups) -> list:
  fullTeachers = []
  halfTeachers = []
  for sub in subjects:
    t = (sub.lecture + countGroups * (sub.practice + sub.laboratory)) / 32
    tmp = math.ceil(t / 5)
    fullTeachers.append(tmp // 2)
    halfTeachers.append(tmp % 2)
  return [halfTeachers, fullTeachers]

Функция *processingLessons* формирует списки предметов и преподавателей.

In [7]:
def processingLessons(vals) -> list:
  subjects = []
  teachers = []
  for sub in vals:
    if not isinstance(sub[1], str):
      subjects.append(Subject(sub[0], sub[1], sub[2], sub[3], sub[4]))
      teachers.append(Teacher(sub[0], sub[9]))
  return [subjects, teachers]

Функция *processingConst* возвращает список константных значений, заполненных в таблице.

In [8]:
def processingConst(vals) -> list:
  countStudents = vals[0][1]
  countGroups = vals[1][1]
  priceM2 = vals[2][1]
  squares = [vals[3][1], vals[4][1], vals[5][1]]
  salarys = [vals[6][1], vals[7][1], vals[8][1], vals[9][1], vals[10][1]]
  program1C = vals[11][1]
  internet = vals[12][1]
  zoom = vals[13][1]
  return [countStudents, countGroups, priceM2, squares, salarys, program1C, internet, zoom]

Функция *totalSalaryTeachers* подсчитывает зарплату для преподавателей за 4 года обучения.

In [9]:
def totalSalaryTeachers(halfTeachers, fullTeachers, teachers, subjects, inflation) -> int:
  tmpSum = 0
  sum = 0
  for i in range(len(teachers)):
    tmpSum += math.ceil((halfTeachers[i] + fullTeachers[i] * 2) * (teachers[i].salary / 2) * subjects[i].semester * 6)
  sum = math.ceil(tmpSum / 2)
  for i in range(3):
    tmpSum = math.ceil((1 + inflation[i][1] / 100) * tmpSum)
    sum += tmpSum
  sum += math.ceil((1 + inflation[3][1] / 100) * tmpSum)
  return math.ceil(sum / 4 / 0.699)

Функция *totalSalaryStaff* подсчитывает зарплату для персонала за 4 года.

In [10]:
def totalSalaryStaff(salarys, inflation) -> int:
  tmpSum = 0
  sum = 0
  for salary in salarys:
    tmpSum += salary
  tmpSum *= 2
  sum = math.ceil(tmpSum / 2)
  for i in range(3):
    tmpSum = math.ceil((1 + inflation[i][1] / 100) * tmpSum)
    sum += tmpSum
  sum += math.ceil((1 + inflation[3][1] / 100) * tmpSum)
  return math.ceil(sum / 4)

Функция *totalSquarePrice* подсчитывает аренду помещения за 4 года.

In [11]:
def totalSquarePrice(halfTeachers, fullTeachers, countGroups, countStudents, squares, salarys, priceM2) -> int:
  teachers = 0
  for i in range(len(halfTeachers)):
    teachers += halfTeachers[i] + fullTeachers[i]
  students = countStudents * countGroups
  staff = len(salarys) * 2
  return int((students * squares[0] + teachers * squares[2]) * priceM2 * 40 + staff * squares[1] * 48)

Функция *zoomPrice* подсчитывает цену на обслуживание Zoom.

In [12]:
def zoomPrice(zoom, countGroups) -> int:
  return int(zoom * 48 * countGroups)

Функция *otherPrice* подсчитывает сумму на обслуживание интернета и программы 1C.

In [13]:
def otherPrice(program1C, internet) -> int:
  return int((program1C + internet * 12) * 4)

In [14]:
subjects, teachers = processingLessons(openexcel('/content/drive/My Drive/Colab Notebooks/Plan.xlsx', 2))

In [15]:
countStudents, countGroups, priceM2, squares, salarys, program1C, internet, zoom = processingConst(openexcel('/content/drive/My Drive/Colab Notebooks/Plan.xlsx', 0))

In [16]:
inflation = openexcel('/content/drive/My Drive/Colab Notebooks/Plan.xlsx', 1)

In [17]:
halfTeachers, fullTeachers = countTeachers(subjects, countGroups)

Ниже представлен подсчет суммы на очное обучение.

In [18]:
fullTime = totalSalaryTeachers(halfTeachers, fullTeachers, teachers, subjects, inflation)
fullTime += totalSalaryStaff(salarys, inflation)
fullTime += totalSquarePrice(halfTeachers, fullTeachers, countGroups, countStudents, squares, salarys, priceM2)
fullTime += otherPrice(program1C, internet)
fullTime = math.ceil(fullTime / (countStudents * countGroups * 2))

Ниже представлен подсчет суммы на дистанционное обучение.

In [19]:
partTime = totalSalaryTeachers(halfTeachers, fullTeachers, teachers, subjects, inflation)
partTime += otherPrice(program1C, internet)
partTime += zoomPrice(zoom, countGroups)
partTime = math.ceil(partTime / (countStudents * countGroups * 2))

Итоговый результат.

In [20]:
print('Цена очного обучения (за семестр):', fullTime, 'руб.')
print('Цена дистанционного обучения (за семестр):',  partTime, 'руб.')
print('Разница цен:', abs(fullTime - partTime), 'руб.')

# Цена очного обучения (за семестр): 174859 руб.
# Цена дистанционного обучения (за семестр): 146213 руб.
# Разница цен: 28646 руб.

Цена очного обучения (за семестр): 174859 руб.
Цена дистанционного обучения (за семестр): 146213 руб.
Разница цен: 28646 руб.
